# Variational Quantum Classifier Feature Map Comparison

Both the first-order and second-order expansion feature maps provided by `qiskit_machine_learning` use $n$ qubits to encode $n$-dim datapoints. However, raw feature vectors can also be directly used in `VQC` circuit constructions, requiring only $log_2(n)$ qubits to encode $n$-dim datapoints. 

### Experiment
Below we compare the classification performance of `VQC` on the [Wine dataset](https://scikit-learn.org/stable/datasets/index.html#wine-dataset) using `RawFeatureVector` and `ZZFeatureMap` feature maps. As you'll see, the former leads to much better testing accuracy using only $2$ qubits, and taking less time. Note that very few training data points are being used.
`VQC` uses [SamplerQNN](https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.neural_networks.SamplerQNN.html#samplerqnn) by default, that's why you don't need to specify a primitive.

In [17]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from qiskit.circuit.library import ZZFeatureMap, TwoLocal

from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit_algorithms.optimizers import SPSA

In [18]:
# function to get sklearn wine dataset, take the desired number of samples and features, scale it and plot it

def wine(training_size, test_size, n_features, plot_data=False):
    """ returns wine dataset """
    data, target = load_wine(return_X_y=True)
    sample_train, sample_test, label_train, label_test = train_test_split(data, target, test_size=test_size, random_state=42)

    std_scale = StandardScaler().fit(sample_train)
    sample_train = std_scale.transform(sample_train)
    sample_test = std_scale.transform(sample_test)

    pca = PCA(n_components=n_features).fit(sample_train)
    sample_train = pca.transform(sample_train)
    sample_test = pca.transform(sample_test)

    samples = np.append(sample_train, sample_test, axis=0)
    minmax_scale = MinMaxScaler(feature_range=(-1, 1)).fit(samples)
    sample_train = minmax_scale.transform(sample_train)
    sample_test = minmax_scale.transform(sample_test)

    sample_train = sample_train[:training_size, :]
    label_train = label_train[:training_size]
    sample_test = sample_test[:training_size, :]
    label_test = label_test[:training_size]

    if plot_data:
        try:
            import matplotlib.pyplot as plt
        except ImportError:
            raise NameError('Matplotlib not installed. Please install it before plotting')
        for k in range(0, 3):
            plt.scatter(sample_train[label_train == k, 0][:training_size],
                        sample_train[label_train == k, 1][:training_size])

        plt.title("PCA dim. reduced Wine dataset")
        plt.show()

    return sample_train, label_train, sample_test, label_test

In [19]:
n_features = 4

train_data, train_labels, test_data, test_labels = wine(
    training_size=20,
    test_size=10, n_features=n_features, plot_data=False
)

Let's try `RawFeatureVector` first:

In [20]:
feature_map = RawFeatureVector(feature_dimension=n_features)

ansatz = TwoLocal(feature_map.num_qubits, ['ry', 'rz'], 'cz', reps=3)
print(ansatz) # this approach requires log2(n_features) = 2 qubits
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    loss="cross_entropy",
    optimizer=SPSA(maxiter=100)
)

vqc.fit(train_data, train_labels)

print(f'Results with RawFeatureVector: train set accuracy -> {vqc.score(train_data, train_labels)}, test set accuracy -> {vqc.score(test_data, test_labels)}.')

     »
q_0: »
     »
q_1: »
     »
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                 ├
«     │  TwoLocal(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7],θ[8],θ[9],θ[10],θ[11],θ[12],θ[13],θ[14],θ[15]) │
«q_1: ┤1                                                                                                 ├
«     └──────────────────────────────────────────────────────────────────────────────────────────────────┘
Results with RawFeatureVector: train set accuracy -> 0.85, test set accuracy -> 0.7.


Now let's try `ZZFeatureMap`:

In [21]:
feature_map = ZZFeatureMap(n_features)

ansatz = TwoLocal(feature_map.num_qubits, ['ry', 'rz'], 'cz', reps=3)
print(ansatz) # this approach requires n_features = 4 qubits

vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    loss="cross_entropy",
    optimizer=SPSA(maxiter=100)
)

vqc.fit(train_data, train_labels)

print(f'Results with ZZFeatureMap: train set accuracy -> {vqc.score(train_data, train_labels)}, test set accuracy -> {vqc.score(test_data, test_labels)}.')

     »
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                                                                 ├
«     │                                                                                                                                                                                                  │
«q_1: ┤1                                                                                                                                                                                                 ├
«     │  TwoLocal(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7],θ[8],θ[9],θ[10],θ[11],θ[12],θ[13],θ[14],θ[15],θ[16],θ[17],θ[18],θ[1

In [22]:
! pip freeze | grep qiskit

qiskit==1.1.0
qiskit-aer==0.14.2
qiskit-algorithms==0.3.0
qiskit-ibm-runtime==0.25.0
qiskit-machine-learning==0.7.2
qiskit-nature==0.7.2
qiskit-nature-pyscf==0.4.0
qiskit-qasm3-import==0.5.0
qiskit-transpiler-service==0.4.5
